# Признаки даты и времени

→ При генерации новых признаков очень ценным может стать временной признак (признак даты и времени).  
Это особый тип данных, с которым приходится сталкиваться в большинстве задач по обработке данных.  
В реальных задачах часто нужно сравнивать даты, выделять день недели или час, вычислять различные интервалы между датами. 

Например, в наших данных об объектах недвижимости есть признак даты продажи — столбец Date.  
Из данного признака можно выделить массу полезной информации, например год, месяц или день недели продажи имущества.  
На рынке недвижимости, как известно, присутствует сезонность: есть периоды, когда недвижимость покупается чаще,  
а есть интервалы времени, когда рынок претерпевает застой, поэтому было бы неплохо учитывать эту сезонность при анализе рынка.

# Формат datetime

В жизни мы видим даты в привычных для нас форматах. Например, запись 01.12.18 обычно означает 1 декабря 2018 года.  
Однако для жителей США эта дата окажется 12 января 2018 года, так как для них привычнее сначала указывать номер месяца, а затем день. 

Многие выгрузки из систем и баз данных имеют свой служебный формат. Например, формат времени из разных систем может отличаться:  
* 2018-11-09 15:45:21;  
* 11/09/2018 3:45:20 PM;  
* 2018-11-09T15:45:21.2984.  

Для всех этих случаев необходимо задавать формат распознавания дат и уметь сравнивать их между собой.  
Для этого был создан единый способ обозначения даты и времени. 

Таким форматом в Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие  
 времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

В наших данных дата записана в виде DD/MM/YYYY, например 3/12/2017. Посмотрим на это:

In [1]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
display(melb_df['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат datetime,  
можно воспользоваться функцией pandas.to_datetime(). В нашем случае в функции нужно указать параметр dayfirst=True,  
который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime,  
передав его в эту функцию:

In [2]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

В результате мы переопределяем признак Date в формат datetime. При этом так как в изначальном варианте время  
не было указано, то и после преобразования оно опускается.

Стоит обратить внимание, что изменился тип данных для столбца Date, теперь его тип — datetime64.  
Рассмотрим несколько возможностей этого типа данных.

# Выделение атрибутов datetime

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

* date — дата;  
* year, month, day — год, месяц, день;  
* time — время;  
* hour, minute, second — час, минута, секунда;  
* dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;  
* day_name — название дня недели;  
* dayofyear — порядковый день года;  
* quarter — квартал (интервал в три месяца).

Например, обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи и понять,  
за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

In [3]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int32
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


В результате обращения к атрибуту datetime melb_df['Date'].dt.year мы получаем объект Series, в котором в качестве  
значений выступают годы продажи объектов недвижимости. Мы можем занести результат в переменную year_sold и далее работать с  
ней как с обычным столбцом Series — вычислять максимум, минимум и модальное значение.

Из результатов видно, что данные представлены за интервал с 2016 по 2017 год и наибольшее количество объектов было продано в 2017 году.

Примечание. Так как модальных значений в столбце может быть несколько, метод mode() возвращает объект Series, даже если мода  
в данных только одна. Чтобы сохранить стилистику вывода информации о годе продажи и выводить только число, а не Series, мы обращаемся  
к результату работы метода mode() по индексу 0.

Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month  
и на этот раз занесём результат в столбец MonthSale, а затем найдём относительную частоту продаж для каждого месяца от общего  
количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях):

In [4]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

MonthSale
5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: proportion, dtype: float64

Из результатов становится ясно, что наибольшее количество продаж недвижимости приходится на май, июль и сентябрь  
(пятый, седьмой и девятый месяцы соответственно). Месяцами застоя при этом являются месяцы — октябрь, февраль  
и январь (десятый, второй и первый месяцы соответственно).

# Работа с интервалами

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно  
вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу  
между датами продаж и заявленной датой, представленной в формате datetime:

In [5]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года.  
Обратите внимание, что данные такого формата относятся к типу timedelta.

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться  
аксессором dt для формата timedelta и извлечь из него атрибут days:

In [6]:
display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи.  
Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим  
в виде столбца AgeBuilding:

In [7]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

Примечание. Обратите внимание, что, так как года кодируются целым числом, в результате мы тоже  
получаем целочисленный столбец — тип int64 (а не timedelta).

На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, так как, зная год постройки здания,  
мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания,  
так как он является более наглядным, а год постройки удалим из таблицы:

melb_df = melb_df.drop('YearBuilt', axis=1)

In [ ]:
# Создать признак WeekdaySale (день недели). НАйти, сколько объектов недвижимости было продано в выходные,
# Результат занесите в переменную weekend_count
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
mask = ((melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)) # Почему-то СБ и ВС это 5 и 6, а не 6 и 7
#display(melb_df['WeekdaySale'].head(10))
#display(melb_df.columns)
display(melb_df[mask])

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates,MonthSale,AgeBuilding,WeekdaySale
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-12-03,2.5,...,1970,Yarra,-37.79960,144.99840,Northern Metropolitan,4019,"-37.7996, 144.9984",12,46,5
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,...,1900,Yarra,-37.80930,144.99440,Northern Metropolitan,4019,"-37.8093, 144.9944",3,117,5
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,2017-03-04,2.5,...,1970,Yarra,-37.79690,144.99690,Northern Metropolitan,4019,"-37.7969, 144.9969",3,47,5
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,2016-06-04,2.5,...,2014,Yarra,-37.80720,144.99410,Northern Metropolitan,4019,"-37.8072, 144.9941",6,2,5
5,5,Abbotsford,129 Charles St,2,h,941000.0,S,Jellis,2016-05-07,2.5,...,1970,Yarra,-37.80410,144.99530,Northern Metropolitan,4019,"-37.8041, 144.9953",5,46,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,2017-08-26,16.7,...,1981,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392,"-37.90562, 145.16761",8,36,5
13576,13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,2017-08-26,6.8,...,1995,NaN,-37.85927,144.87904,Western Metropolitan,6380,"-37.85927, 144.87904",8,22,5
13577,13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,2017-08-26,6.8,...,1997,NaN,-37.85274,144.88738,Western Metropolitan,6380,"-37.85274, 144.88738",8,20,5
13578,13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,2017-08-26,6.8,...,1920,NaN,-37.85908,144.89299,Western Metropolitan,6380,"-37.85908, 144.89299",8,97,5


In [28]:
# Загрузка данных об НЛО
nlo_data =pd.read_csv('data/nlo.csv', sep=',')
nlo_data['Time'] = pd.to_datetime(nlo_data['Time'])
nlo_data.head(3)

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,1930-06-01 22:00:00
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00


In [26]:
# В каком году наблюдается наибольшее количество случаев наблюдения НЛО в США?
display(nlo_data['Time'].dt.year.mode())

0    1999
Name: Time, dtype: int32

In [39]:
# Найдите средний интервал времени (в днях) между двумя последовательными случаями наблюдения НЛО в штате Невада (NV).
nlo_data['Date'] = nlo_data['Time'].dt.date
#nlo_data.head(3)
mask = (nlo_data['State'] == 'NV')
display(nlo_data[mask])
nlo_diff = nlo_data[mask]['Date'].diff().dt.days.mean()
display(nlo_diff)

,City,Colors Reported,Shape Reported,State,Time,Date
76,Las Vegas,NaN,DISK,NV,1947-07-15 10:00:00,1947-07-15
172,Nellis AFB,NaN,DISK,NV,1952-02-17 18:00:00,1952-02-17
565,Fallon,NaN,OVAL,NV,1959-09-15 00:00:00,1959-09-15
566,Goldfield,NaN,LIGHT,NV,1959-09-15 01:00:00,1959-09-15
613,NaN,NaN,DISK,NV,1960-07-01 12:00:00,1960-07-01
...,...,...,...,...,...,...
17447,Laughlin,NaN,FORMATION,NV,2000-09-16 22:00:00,2000-09-16
17567,Las Vegas,NaN,SPHERE,NV,2000-09-30 22:25:00,2000-09-30
17617,Las Vegas,RED YELLOW,OTHER,NV,2000-10-06 20:25:00,2000-10-06
17890,Reno,NaN,TRIANGLE,NV,2000-11-07 02:15:00,2000-11-07


np.float64(68.92932862190813)